In [25]:
import pandas as pd

import numpy as np
import tensorflow as tf
import networkx as nx

wd = '/Users/ewenwang/Documents/practice_data/conversion_rate/'
data = 'round1_ijcai_18_train_20180301.txt'

target = 'is_trade'
features = ['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 
            'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
            'shop_review_num_level', 'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description']

sourcetarget = ['user_id', 'item_id']

nodecol = 'user_id'

In [ ]:
data = pd.read_csv(wd+data, sep=' ')

In [26]:
data = data.head(2000)

In [27]:
edgelist = pd.DataFrame()
edgelist['user_id'] = data.user_id
edgelist['item_id'] = data.item_id

df_target = data[sourcetarget+[target]].groupby(sourcetarget ,as_index=False).sum()
df_features = data[sourcetarget+features].groupby(sourcetarget, as_index=False).mean()

df = df_target.merge(df_features, how='left', on=sourcetarget)

In [67]:
df.head()

,user_id,item_id,is_trade,user_gender_id,user_age_level,user_occupation_id,user_star_level,item_price_level,item_sales_level,item_collected_level,item_pv_level,shop_review_num_level,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description
0,8078017541935668,4215883534709645310,0,0,1002,2005,3006,7,11,13,19,21,0.99704,0.979661,0.979589,0.975442
1,8078017541935668,8824610546121211352,0,0,1002,2005,3006,7,11,12,19,21,0.99704,0.979661,0.979589,0.975442
2,18242482054767926,5323767931396822716,0,0,1003,2005,3004,7,13,14,19,16,1.00000,0.967400,0.967237,0.983510
3,23297531048455633,4215883534709645310,0,0,1003,2002,3006,7,11,13,19,21,0.99704,0.979661,0.979589,0.975442
4,23297531048455633,8824610546121211352,0,0,1003,2002,3006,7,11,12,19,21,0.99704,0.979661,0.979589,0.975442


In [50]:
def onehot_encoder(target_ls):
    classes = set(target_ls)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    target_onehot = np.array(list(map(classes_dict.get, target_ls)), dtype=np.int32)
    return target_onehot, [*classes]

def adjacency_normalizer(adj):
    adj = adj + np.eye(adj.shape[0])
    d = np.diagflat(np.power(np.array(adj.sum(1)), -1))
    a_norm = d.dot(adj)
    return a_norm

def graph_convolution(a, x, w, b):
    with tf.name_scope('gcn_layer'):
        kernel = tf.add(tf.matmul(a, tf.matmul(x, w)), b)
        return tf.nn.leaky_relu(kernel)

In [51]:
g = nx.from_pandas_edgelist(edgelist, 
                            source=edgelist.columns[0], 
                            target=edgelist.columns[1], 
                            create_using=nx.MultiGraph())

a = adjacency_normalizer(nx.to_numpy_matrix(g))

In [78]:
len(g.nodes)

1747

In [145]:
df_node = pd.DataFrame({nodecol: g.nodes()})
df_node = pd.merge(df_node, df, how='left', on=nodecol, sort=False)

df_node = df_node[np.isfinite(df_node[target])]

df_node = df_node.head(a.shape[0])    # trick!

x = df_node[features]
y, classes = onehot_encoder(list(df_node[target]))

In [146]:
df_node.shape

(1747, 16)

In [147]:
a.shape[0]

1747

In [148]:
n_nodes = g.number_of_nodes()
n_featues = len(features)
n_classes = len(classes)

In [149]:
keep_prob = 0.5
learning_rate = 0.001

n_epochs = 100
verbose = 10
loss_ls = []

In [159]:
adjacency = tf.placeholder(tf.float32, shape=(n_nodes, n_nodes), name='adjacency')
feature = tf.placeholder(tf.float32, shape=(n_nodes, n_featues), name='feature')
label = tf.placeholder(tf.float32, shape=(n_nodes, n_classes), name='label')

weights = tf.Variable(tf.random_normal([n_featues, n_classes], seed=0), name='weights')
bais = tf.Variable(tf.random_normal([n_nodes, n_classes], seed=0), name='bais')
    
hidden1 = graph_convolution(a=adjacency, x=feature, w=weights, b=bais)
hidden2 = tf.nn.dropout(hidden1, keep_prob=keep_prob)
# output = tf.nn.softmax(logits=hidden2, axis=n_classes)
output = tf.nn.softmax(logits=hidden2, axis=-1)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=label))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [160]:
predictions = None

In [161]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        training_op.run(feed_dict={adjacency: a, feature: x, label: y})
        if verbose <= 0:
            pass
        elif epoch % verbose == 0:
            err = loss.eval(feed_dict={adjacency: a, feature: x, label: y})
            loss_ls.append(err)
            print(epoch, '\tloss: %.6f' %err)
    predictions = output.eval(feed_dict={adjacency: a, feature: x, label: y})

0 	loss: 0.893691
10 	loss: 0.909639
20 	loss: 0.920780
30 	loss: 0.907017
40 	loss: 0.910829
50 	loss: 0.884805
60 	loss: 0.920550
70 	loss: 0.876733
80 	loss: 0.915213
90 	loss: 0.907314


In [155]:
pred = pd.DataFrame(predictions, columns=classes)
pred.head()

,0.0,1.0
0,0.0,1.000000e+00
1,1.0,1.952767e-21
2,0.5,5.000000e-01
3,0.0,1.000000e+00
4,0.5,5.000000e-01


In [162]:
from Report import Report

In [164]:
rpt = Report(classifier=None, train=df_node, test=df_node, target=target, predictors=features)

In [166]:
rpt.train_predprob = predictions[1]
rpt.test_predprob = predictions[1]

rpt.train_predictions = np.where(rpt.train_predprob>=0.5, 1, 0)
rpt.test_predictions = np.where(rpt.test_predprob>=0.5, 1, 0)

In [168]:
rpt.ROC()

ValueError: Found input variables with inconsistent numbers of samples: [1747, 2]